In [12]:
from model import TransformerDecoder

from dataset import TextDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# we want to train our tokenizer on the whole dataset

text_dataset = TextDataset(data_file="stories.txt",
                            train=True,
                           sp_model_prefix="sp_model",
                           vocab_size=4000,)

trainer_interface.cc(378) LOG(WARNING) Found too long line (4928 > 4192).
trainer_interface.cc(380) LOG(WARNING) Too long lines are skipped in the training.
trainer_interface.cc(381) LOG(WARNING) The maximum length can be changed with --max_sentence_length=<size> flag.
trainer_interface.cc(122) LOG(WARNING) Too many sentences are loaded! (5595306), which may slow down training.
trainer_interface.cc(124) LOG(WARNING) Consider using --input_sentence_size=<size> and --shuffle_input_sentence=true.
trainer_interface.cc(127) LOG(WARNING) They allow to randomly sample <size> sentences from the entire corpus.


In [52]:

!python3 parse_stories.py

In [85]:
!head -n 128 short_stories.txt > train.txt
!tail -n 10 short_stories.txt > test.txt

!head -n 128 stories.txt > large_train.txt
!tail -n 64 stories.txt > large_test.txt

In [86]:
import torch
from trainloop import train
from torch.utils.data import DataLoader
from utils import collate_fn

BATCH_SIZE = 64
NUM_EPOCHS = 300

train_set = TextDataset(data_file="large_test.txt", train=True, sp_model_prefix="sp_model", vocab_size=4000,)
valid_set = TextDataset(data_file="large_test.txt", train=True, sp_model_prefix="sp_model", vocab_size=4000,)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [82]:
train_set[1]

(tensor([   1,  136,  141,    6,  115, 3989,  118,   30,    6,  113,  179,  127,
           62, 3983,   44,  169,   13, 1264,   75,   53, 3775,   15,  284,  468,
         1159, 3983,  104,   70, 3989,   62, 3993, 3976,  246,  437,   53,    6,
          271,  789,   13,  917, 3983,   48,   30,  102,  650,   15,  304,   62,
          417,  269,    6, 1413, 3983,   93,  104,   70, 3989,   62,  159,  314,
           15,   48,   30,  135,  975, 3983,    8,  579,   30, 2377,   15,  503,
         2550, 3979, 3983,   62,  280, 3993, 3969,  124,   13,  859,   19,   53,
          468,  789, 3989,  102,   44,  386,   13,  708,  399, 3983,   62, 3993,
         3976,  246,  126,   89,   44,   30,  222,   15,  221,   53,  980, 3983,
           62,  372,   53,   89,   44,  280, 3993, 3969,  124,   13,  201,  314,
           15,  351,   53,  789, 1168, 3983,   53,  246,  307,   15]),
 242)

In [83]:
import wandb
wandb.login()

wandb.init(project="LLM-Homework")

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerDecoder(vocab_size=text_dataset.vocab_size,
                            embed_dim=64,
                            n_blocks=2,
                            n_head=2,
                            ff_dim=64,
                            text_dataset=text_dataset,)
print(f"Number of parameters in the model: {sum(p.numel() for p in model.parameters())}")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.98), eps=1e-9)
criterion = torch.nn.CrossEntropyLoss(ignore_index=text_dataset.pad_id)

train_losses, test_losses = train(model=model, optimizer=optimizer, criterion=criterion,
                                  train_loader=train_loader, test_loader=val_loader, num_epochs=NUM_EPOCHS)

Number of parameters in the model: 566432


test_loss,██▇▇▇▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
training_loss,██▇▇▇▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
test_loss,3.89335
training_loss,3.97092
